# T2. 开发自己的 task

`fastie` 中集成了一些常用的 `task`，但是并不是所有的任务都能满足你的需求，因此你可以自己开发自己的 `task`.
开发自己的 `task` 可以选择继承已有任务进行修改, 或者继承 `BaseTask` 类全新开发.

## 1. task 的生命周期

`fastie` 中, 每个 `task` 的 `run` 方法都遵守一套固定的流程(我们将其称为 `fastie` 的生命周期),
其中流程的每个阶段都有固定的任务目标. 换而言之, `task` 的每个生命周期方法都有固定的参数输入和期望的
返回值. 例如, `task` 的 `on_setup_model` 方法的任务目标是模型的搭建, 因此该方方法期待返回一个拥有
`train_step`, `evaluation_step` 和 `infer_step` 方法的 `model` 对象. 因此, 在实现自己
的 `task` 的过程中, 你需要遵守 `fastie` 的生命周期, 并且在每个生命周期方法中返回期望的对象.

`fastie` 的生命周期如下:

<img src="./figures/T3-task-life.jpg" />

### 1.1 on_generate_and_check_tag_vocab

`on_generate_and_check_tag_vocab` 方法的功能为, 根据输入原始数据集 `data_bundle` 生成
标签词典 `tag_vocab`, 并将生成的词典和可能存在的从模型加载到的词典对比检查.

该方法的输入为原始数据集 `data_bundle` 和 `checkpoint` 信息 `state_dict`. 该方法应返回一个
`fastNLP.Vocabulary` 的变量或数组 (存在多个标签值的情况).

如上流程图所示, 该方法为非必要重写的方法, `task` 的基类 `Basetask` 已经实现了该方法的基本逻辑.
在有特殊需求的情况下可以重新定义该方法.

### 1.2 on_dataset_preprocess

`on_dataset_preprocess` 方法为将原始数据集 `data_bundle` 进行预处理的方法, 包括把原始的
`tag` 通过上一步生成的 `tag_vocab` 进行转换为可以计算的 `id`, 以及将 `token` 转换为 `id` 等.

该方法的输入为原始数据集 `data_bundle`, 上一步产生的一个或多个 `tag_vocab`, 以及 `checkpoint`
信息 `state_dict`. 该方法应返回处理过后的 `data_bundle`, 数据类型为 `fastNLP.io.DataBundle`.

如上流程图所示, 该方法为必须重写的方法, `task` 的基类 `Basetask` 未实现该方法. 直接调用会导致异常.

### 1.3 on_setup_model

`on_setup_model` 方法为模型的实现方法, 包括模型的创建和初始化.

该方法的输入为上一步处理过后的 `data_bundle`, `on_generate_and_check_tag_vocab` 的输出
`tag_vvocab`, 以及 `checkpoint` 信息 `state_dict`. 该方法的输出为一个拥有 `train_step`,
`evaluation_step` 和 `infer_step` 方法的 `model` 对象.

如上流程图所示, 该方法为必须重写的方法, `task` 的基类 `Basetask` 未实现该方法. 直接调用会导致异常.

### 1.4 on_setup_optimizers

`on_setup_optimizers` 方法为优化器的实现方法, 包括优化器的创建和初始化.

该方法的输入为 `on_dataset_preprocess` 的输出 `data_bundle`, `on_setup_model` 的输出 `model`,
`on_generate_and_check_tag_vocab` 的输出 `tag_vocab`, 以及 `checkpoint` 信息 `state_dict`.
该方法的输出为训练所需的优化器, 可以是单独的一个优化器实例，也可以是多个优化器组成的 `List`.

如上流程图所示, 该方法为必须重写的方法, `task` 的基类 `Basetask` 未实现该方法. 直接调用会导致异常.

### 1.5 on_setup_dataloader

`on_setup_dataloader` 方法为数据加载器的实现方法, 包括数据加载器的创建和初始化.

该方法的输入为 `on_dataset_preprocess` 的输出 `data_bundle`, `on_setup_model` 的输出 `model`,
`on_generate_and_check_tag_vocab` 的输出 `tag_vocab`, 以及 `checkpoint` 信息 `state_dict`.

该方法的输出需要根据当前的控制器判断, 当前的控制器可以通过 `fastie.envs.get_flag` 方法获得,
可能的取值包括: `train`, `eval`, `infer`. 需要根据当前的控制器来判断取用 `data_bundle` 的哪个 `split.
该方法的输出可以是一个可迭代的 `dataloader` 组成的 `dict`, 其中当 `flag` 为 `train` 时, `key` 为 `train`
将被用于训练集, 其他的会被用于验证集; 当 `flag` 为 `eval`, `infer` 时, 所有的 `dataloader` 会被用作测试集
和推理集.

如上流程图所示, 该方法为非必要重写的方法, `task` 的基类 `Basetask` 已经实现了该方法的基本逻辑.
在有特殊需求的情况下可以重新定义该方法.

### 1.6 on_setup_callbacks

`on_setup_callbacks` 创建训练过程中的回调项.

该方法的输入为 `on_dataset_preprocess` 的输出 `data_bundle`, `on_setup_model` 的输出 `model`,
`on_generate_and_check_tag_vocab` 的输出 `tag_vocab`, 以及 `checkpoint` 信息 `state_dict`.

该方法的输出为一个 `fastNLP.Callback` 对象或者 `fastNLP.Callback` 的列表. 具体可参照
[fastNLP.callbacks](http://www.fastnlp.top/docs/fastNLP/master/api/core.html#callbacks).

如上流程图所示, 该方法为非必要重写的方法, `task` 的基类 `Basetask` 默认不实现任何回调.

### 1.7 on_setup_metrics

`on_setup_metrics` 创建验证过程中的评估指标.

该方法的输入为 `on_dataset_preprocess` 的输出 `data_bundle`, `on_setup_model` 的输出 `model`,
`on_generate_and_check_tag_vocab` 的输出 `tag_vocab`, 以及 `checkpoint` 信息 `state_dict`.

该方法的返回结果应该为一个字典, 例如: ``{"acc1": Accuracy(), "acc2": Accuracy()}``.

目前我们支持的 ``metric`` 的种类有以下几种：

1. fastNLP 的 ``metric``: 详见 [fastNLP.metrics](http://www.fastnlp.top/docs/fastNLP/master/api/core.html#metrics);
2. torchmetrics;
3. allennlp.training.metrics;
4. paddle.metric;

如上流程图所示, 该方法为非必要重写的方法, `task` 的基类 `Basetask` 默认不实现任何 `metric`.
注意: 如果要使用 `fastie` 的 `load_best_model` 或 `topk` 等必需 `metric` 的特性, 则必须重写
该方法.

### 1.8 on_setup_extra_fastnlp_parameters

`on_setup_extra_fastnlp_parameters` 方法为 `fastNLP` 的一些额外参数的设置.

该方法的输入为 `on_dataset_preprocess` 的输出 `data_bundle`, `on_setup_model` 的输出 `model`,
`on_generate_and_check_tag_vocab` 的输出 `tag_vocab`, 以及 `checkpoint` 信息 `state_dict`.

该方法的返回结果应该为一个字典, 对应 `fastNLP.Trainer` 或 `fastNLP.Evaluator` 的参数,
参见:
[fastNLP.Trainer](http://www.fastnlp.top/docs/fastNLP/master/api/generated/fastNLP.core.Trainer.html#fastNLP.core.Trainer),
[fastNLP.Evaluator](http://www.fastnlp.top/docs/fastNLP/master/api/generated/fastNLP.core.Evaluator.html#fastNLP.core.Evaluator)

如上流程图所示, 该方法为非必要重写的方法, `task` 的基类 `Basetask` 默认不添加任何额外参数.

### 1.9 on_get_state_dict

`on_get_state_dict` 方法为获取 `checkpoint` 变量 `state_dict` 的方法.

该方法的输入为 `on_dataset_preprocess` 的输出 `data_bundle`, `on_setup_model` 的输出 `model`,
以及 `on_generate_and_check_tag_vocab` 的输出 `tag_vocab`.

该方法的输出应与前面所有的生命周期方法的参数 `state_dict` 格式一直.

如上流程图所示, 该方法为非必要重写的方法, `task` 的基类 `Basetask` 默认保存 `model` 的
`state_dict` 以及 `tag_vocab`。

## 2. 实战

`fastie` 默认提供了使用预训练的 `BERT` 模型进行 `NER` 任务的 `task` 类 `BertNER`, 该任务中
默认使用的优化器为 `torch.optim.Adam`. 因此, 我们可以通过重写 `on_setup_optimizer` 方法来
实现使用 `AdamW` 优化器的 `NER` 任务.

```python
from fastie.tasks import BertNER
from torch.optim import AdamW

class BertNERAdamW(BertNER):
    def on_setup_optimizer(self, model, tag_vocab, data_bundle, state_dict=None):
        return AdamW(model.parameters(), lr=1e-3)
```